### 文本生成

#### 1. 先將輸入字元 Token 並轉換成 PyTorch Tensor 物件：
* Tensor 的字典：
    * `input_ids`: 分詞的結果。
    * `attention_mask`: 用來遮罩非輸入部分。

In [1]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM as ModelCls
from transformers import AutoTokenizer as TkCls

model_path = "TheBloke/Llama-2-7b-chat-fp16"
model: ModelCls = ModelCls.from_pretrained(
    model_path,
    device_map="auto",
    low_cpu_mem_usage=True,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
)
tk: TkCls = TkCls.from_pretrained(model_path)
tokens = tk("Hello, ", return_tensors="pt")
print(tokens) # Tokenize 結果
"""
Output:
{
    "input_ids": tensor([[1, 15043, 29892, 29871]]),
    "attention_mask": tensor([[1, 1, 1, 1]]),
}
"""

/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.02it/s]
/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. 

{'input_ids': tensor([[    1, 15043, 29892, 29871]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


'\nOutput:\n{\n    "input_ids": tensor([[1, 15043, 29892, 29871]]),\n    "attention_mask": tensor([[1, 1, 1, 1]]),\n}\n'

#### 處理 `input_ids`
一開始只需要關注 `input_ids` ，先把 `attention_mask` 當成模型的必要輸入。
* 模型與 Tensor 必須放在同一個裝置中才能做運算。
    * 例如，放在 CPU 上的模型只能跟 CPU 上的 Tensor 運算。
    * 這裡模型放在 GPU 上，所以要用 `.to("cuda")` 將整個分詞放進 GPU 裡面。

In [2]:
input_ids = tokens["input_ids"].to("cuda")
print(input_ids)
# tensor([[    1, 15043, 29892, 29871]], device='cuda:0')

tensor([[    1, 15043, 29892, 29871]], device='cuda:0')


#### 2. 開始使用 model.generate 進行文字產生：
* `model.generate` 會回傳產生的 Token ID，必須用 Tokenizer 進行 Decode 才能得到文字版的輸出。
* `max_new_tokens` 是用來設定要輸出的 Token 數量。
    * 在 Transformer Decoder 裡面，輸出的Token數量越多，佔用的GPU記憶體就會越多，生成需要的時間理所當然的也比較久。

In [6]:
output = model.generate(input_ids, max_new_tokens=32)
print(tk.batch_decode(output))
# <s> Hello, I am a beginner in Python ...

['</s></s><s> Hello,  I am trying to create a simple web application that allows users to upload a file', "<s> Hi, my name is Sarah and I'm a 30-year-old woman from the"]


#### 3. 一次做很多個文字生成，俗稱**批次推論 (Batch Inference)**：
* **注意**，因為 Llama Tokenizer 沒有預設填充用的 Padding Token，所以需要幫它指定一個，才能做填充。

In [4]:
tk.pad_token = tk.eos_token  # LlamaTokenizer 沒有 Padding Token
prompt = ["Hello, ", "Hi, my name is"]
tokens = tk(prompt, return_tensors="pt", padding=True)
input_ids = tokens["input_ids"].to("cuda")

outputs = model.generate(input_ids, max_new_tokens=16)
print(tk.batch_decode(outputs))

['</s></s><s> Hello,  I am trying to create a simple web application that allows users to upload a file', "<s> Hi, my name is Sarah and I'm a 30-year-old woman from the"]


#### 有時會出現輸出結果怪怪的結果：

* 因為 Transformers Decoder 是自回歸解碼 (Autoregressive) 的關係，放在越右邊的 Token 通常會對下一個生成的 Token 有越顯著的影響。
    * 若是將 Padding Token 都放右邊，模型看到一堆 Padding Token 後也不知道要輸出什麼。
    * 所以在進行批次生成時，記得要把 Padding Token 放在左邊，可以在初始化 Tokenizer 時設定 `padding_side` 這個參數。

* 怪怪的結果：
    ```
    [
        "<s> Hello, </s></s>0000000000000000",
        "<s> Hi, my name is [Your Name] and I am a [Your Profession] ...",
    ]
    ```
* 修正後的結果：
    ```
    [
        "</s></s><s> Hello, I am a 35 year old woman ...",
        "<s> Hi, my name is [Your Name] and I am a [Your Profession] ...",
    ]
    ```

In [5]:
tk: TkCls = TkCls.from_pretrained(model_path, padding_side="left")
tk.pad_token = tk.eos_token  # LlamaTokenizer 有 Padding Token
prompt = ["Hello, ", "Hi, my name is"]
tokens = tk(prompt, return_tensors="pt", padding=True)
input_ids = tokens["input_ids"].to("cuda")

outputs = model.generate(input_ids, max_new_tokens=16)
print(tk.batch_decode(outputs))

['</s></s><s> Hello,  I am trying to create a simple web application that allows users to upload a file', "<s> Hi, my name is Sarah and I'm a 30-year-old woman from the"]
